In [1]:
import joblib

In [2]:
import gdown
import pandas as pd

# Define the URL of the CSV file
file_id = '1WClX4TtAVny-bz7AI1VNrqeGlnzyXiam'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file using gdown
output = 'sampled_data_full.csv'
gdown.download(url, output, quiet=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(output)

# Display the last few rows of the DataFrame
df.tail()

Downloading...
From (original): https://drive.google.com/uc?id=1WClX4TtAVny-bz7AI1VNrqeGlnzyXiam
From (redirected): https://drive.google.com/uc?id=1WClX4TtAVny-bz7AI1VNrqeGlnzyXiam&confirm=t&uuid=7f9894f8-1a44-4ff4-8d9f-f6c1b19af065
To: /Users/jinluwang/Downloads/Do work here/Challenges/Flight Delay/predictive_flight_delays_2/notebooks/sampled_data_full.csv
100%|██████████| 268M/268M [00:07<00:00, 33.7MB/s] 


,MONTH,DAY_OF_WEEK,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,...,PLANE_AGE,DEPARTING_AIRPORT,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND
1297807,6,5,0,0800-0859,7,1,37,150,JetBlue Airways,12427,...,8,Logan International,42.364,-71.006,NONE,0.0,0.0,0.0,77.0,14.32
1297808,10,3,0,1100-1159,4,3,5,76,SkyWest Airlines Inc.,5961,...,0,Washington Dulles International,38.942,-77.458,Detroit Metro Wayne County,0.0,0.0,0.0,96.0,6.04
1297809,5,3,0,1900-1959,2,5,9,70,Comair Inc.,2558,...,15,Bradley International,41.939,-72.683,Ronald Reagan Washington National,0.0,0.0,0.0,68.0,6.93
1297810,6,3,1,0700-0759,4,2,45,76,SkyWest Airlines Inc.,21878,...,4,Stapleton International,39.774,-104.880,Billings Logan International,0.0,0.0,0.0,83.0,7.83
1297811,6,2,0,0700-0759,3,1,13,173,United Air Lines Inc.,4615,...,6,Kansas City International,39.298,-94.714,NONE,0.0,0.0,0.0,84.0,9.40


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [4]:
# Sample 20% of the data
data_sample = df.sample(frac=0.2, random_state=42)

# Define features and target
features = ['MONTH', 'DAY_OF_WEEK', 'DEP_TIME_BLK',
            'TMAX', 'AWND','AIRPORT_FLIGHTS_MONTH','AIRLINE_FLIGHTS_MONTH','AIRLINE_AIRPORT_FLIGHTS_MONTH','PRCP']
print(len(features))
# Prepare the data
X = data_sample[features]
y = data_sample['DEP_DEL15']

9


In [5]:
data_sample[features]

,MONTH,DAY_OF_WEEK,DEP_TIME_BLK,TMAX,AWND,AIRPORT_FLIGHTS_MONTH,AIRLINE_FLIGHTS_MONTH,AIRLINE_AIRPORT_FLIGHTS_MONTH,PRCP
738784,8,4,1300-1359,92.0,3.13,15106,22671,1061,0.00
1222149,5,5,0800-0859,65.0,5.37,12849,113709,948,0.24
63596,1,4,1600-1659,44.0,4.70,6023,62105,559,0.00
1183182,7,6,0700-0759,89.0,7.83,1598,72721,731,0.00
1003874,3,7,1400-1459,62.0,14.32,1375,114119,535,0.00
...,...,...,...,...,...,...,...,...,...
918134,10,7,1000-1059,88.0,10.07,25833,71625,1405,0.97
772057,2,5,0600-0659,21.0,5.14,22752,43512,5161,0.02
51002,10,2,2000-2059,79.0,8.72,20449,24198,8189,0.57
375745,5,4,0800-0859,83.0,10.51,12037,22451,5958,0.00


In [6]:
X.columns

Index(['MONTH', 'DAY_OF_WEEK', 'DEP_TIME_BLK', 'TMAX', 'AWND',
       'AIRPORT_FLIGHTS_MONTH', 'AIRLINE_FLIGHTS_MONTH',
       'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'PRCP'],
      dtype='object')

In [7]:
X.iloc[0]

MONTH                                    8
DAY_OF_WEEK                              4
DEP_TIME_BLK                     1300-1359
TMAX                                  92.0
AWND                                  3.13
AIRPORT_FLIGHTS_MONTH                15106
AIRLINE_FLIGHTS_MONTH                22671
AIRLINE_AIRPORT_FLIGHTS_MONTH         1061
PRCP                                   0.0
Name: 738784, dtype: object

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 259562 entries, 738784 to 207473
Data columns (total 9 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   MONTH                          259562 non-null  int64  
 1   DAY_OF_WEEK                    259562 non-null  int64  
 2   DEP_TIME_BLK                   259562 non-null  object 
 3   TMAX                           259562 non-null  float64
 4   AWND                           259562 non-null  float64
 5   AIRPORT_FLIGHTS_MONTH          259562 non-null  int64  
 6   AIRLINE_FLIGHTS_MONTH          259562 non-null  int64  
 7   AIRLINE_AIRPORT_FLIGHTS_MONTH  259562 non-null  int64  
 8   PRCP                           259562 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 19.8+ MB


In [9]:
label_enc = LabelEncoder()
encoded_time = label_enc.fit_transform(X['DEP_TIME_BLK'])
joblib.dump(label_enc, "../trained_modules/label_encoder.pkl")

['../trained_modules/label_encoder.pkl']

In [10]:
X =X.drop(columns=["DEP_TIME_BLK"])

In [11]:
X["ENCODED_DEP_TIME_BLK"] = encoded_time

In [12]:
X.isna().sum()

MONTH                            0
DAY_OF_WEEK                      0
TMAX                             0
AWND                             0
AIRPORT_FLIGHTS_MONTH            0
AIRLINE_FLIGHTS_MONTH            0
AIRLINE_AIRPORT_FLIGHTS_MONTH    0
PRCP                             0
ENCODED_DEP_TIME_BLK             0
dtype: int64

In [13]:
X.columns

Index(['MONTH', 'DAY_OF_WEEK', 'TMAX', 'AWND', 'AIRPORT_FLIGHTS_MONTH',
       'AIRLINE_FLIGHTS_MONTH', 'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'PRCP',
       'ENCODED_DEP_TIME_BLK'],
      dtype='object')

In [14]:
# Handle categorical variables
# categorical_features = ['CARRIER_NAME', 'DEPARTING_AIRPORT', 'PREVIOUS_AIRPORT']
# X = pd.get_dummies(X, columns=categorical_features, drop_first=True)

# Handle 'DEP_TIME_BLK' separately
# X['DEP_TIME_BLK'] = X['DEP_TIME_BLK'].astype('category').cat.codes

# Make sure all columns are numeric
# X = X.apply(pd.to_numeric, errors='coerce')

# Drop any rows with NaN values
# X.dropna(inplace=True)
# y = y[X.index]

In [15]:
# Use pandas for categorical to numerical conversion.

In [16]:
# Normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [17]:
joblib.dump(scaler, "../trained_modules/standard_scaler.pkl")

['../trained_modules/standard_scaler.pkl']

In [18]:
scaler.get_feature_names_out()

array(['MONTH', 'DAY_OF_WEEK', 'TMAX', 'AWND', 'AIRPORT_FLIGHTS_MONTH',
       'AIRLINE_FLIGHTS_MONTH', 'AIRLINE_AIRPORT_FLIGHTS_MONTH', 'PRCP',
       'ENCODED_DEP_TIME_BLK'], dtype=object)

In [19]:
# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [20]:
# Build the TensorFlow Neural Network
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

/Users/jinluwang/anaconda3/envs/mlenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
6490/6490 ━━━━━━━━━━━━━━━━━━━━ 3s 422us/step - accuracy: 0.8088 - loss: 0.4677 - val_accuracy: 0.8153 - val_loss: 0.4542
Epoch 2/20
6490/6490 ━━━━━━━━━━━━━━━━━━━━ 3s 411us/step - accuracy: 0.8103 - loss: 0.4595 - val_accuracy: 0.8160 - val_loss: 0.4524
Epoch 3/20
6490/6490 ━━━━━━━━━━━━━━━━━━━━ 3s 415us/step - accuracy: 0.8111 - loss: 0.4582 - val_accuracy: 0.8163 - val_loss: 0.4518
Epoch 4/20
6490/6490 ━━━━━━━━━━━━━━━━━━━━ 3s 421us/step - accuracy: 0.8113 - loss: 0.4572 - val_accuracy: 0.8163 - val_loss: 0.4519
Epoch 5/20
6490/6490 ━━━━━━━━━━━━━━━━━━━━ 3s 416us/step - accuracy: 0.8126 - loss: 0.4556 - val_accuracy: 0.8150 - val_loss: 0.4512
Epoch 6/20
6490/6490 ━━━━━━━━━━━━━━━━━━━━ 3s 415us/step - accuracy: 0.8123 - loss: 0.4547 - val_accuracy: 0.8161 - val_loss: 0.4501
Epoch 7/20
6490/6490 ━━━━━━━━━━━━━━━━━━━━ 3s 408us/step - accuracy: 0.8126 - loss: 0.4547 - val_accuracy: 0.8162 - val_loss: 0.4515
Epoch 8/20
6490/6490 ━━━━━━━━━━━━━━━━━━━━ 3s 414us/step - accuracy: 0.8112 -

In [22]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy:', test_acc)

1623/1623 ━━━━━━━━━━━━━━━━━━━━ 0s 253us/step - accuracy: 0.8144 - loss: 0.4512

Test accuracy: 0.8166740536689758


In [23]:
# Predict the labels of the test set
y_pred = (model.predict(X_test) > 0.5).astype("int32")

1623/1623 ━━━━━━━━━━━━━━━━━━━━ 0s 245us/step


In [24]:
joblib.dump(model, "../trained_modules/tf_1.0.pkl")

['../trained_modules/tf_1.0.pkl']